# COMS 4995_002 Deep Learning Assignment 2
Due on Monday, Oct 30, 11:59pm

This assignment can be done in groups of at most 3 students. Everyone must submit on Courseworks individually.

Write down the UNIs of your group (if applicable)

Member 1: Animesh Anant Sharma, aas2325

Member 2: Name, UNI

Member 3: Name, UNI

In [26]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras import optimizers
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.constraints import maxnorm
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
import time, pickle


In [20]:
# Helper functions

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [21]:
# Functions to load data

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr / 255.0
        images.append(img_arr)
    X = np.array(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [6]:
# Load the data
data_root_path = '/home/animesh/deeplearning/cifar10-hw1/'
X1, y = get_train_data(data_root_path) # this may take a few minutes
X_test1 = get_images(data_root_path + 'test')
print('Data loading done')

{'horse': 7, 'cat': 3, 'dog': 5, 'ship': 8, 'bird': 2, 'deer': 4, 'airplane': 0, 'truck': 9, 'automobile': 1, 'frog': 6}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [22]:
X = X1.astype('float32')
X_test = X_test1.astype('float32')
X_train = X[:45000]
y_train = y[:45000]
X_val = X[45000:]
y_val = y[45000:]
Y_train = np_utils.to_categorical(y_train, 10)
Y_val = np_utils.to_categorical(y_val, 10)
print('X shape:', X.shape)
print('X_train shape:', X_train.shape)
print('X_val shape:', X_val.shape)
print('X_test shape:', X_test.shape)
print('Y_train shape:', Y_train.shape)
print(Y_train[0])
print(Y_val[0])

X shape: (50000, 32, 32, 3)
X_train shape: (45000, 32, 32, 3)
X_val shape: (5000, 32, 32, 3)
X_test shape: (10000, 32, 32, 3)
Y_train shape: (45000, 10)
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]


##  Simple convolutional neural network


In [49]:
epochs = 15
b_s = 32
learning_rate = 0.01

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

sgd = optimizers.SGD(lr=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#print(model.summary())

In [50]:
tensorboard_callback = TensorBoard(log_dir='/tmp/cifar10', histogram_freq=2, write_graph=True, write_images=False, write_grads=True)

In [51]:
h = model.fit(
    X_train,
    Y_train,
    batch_size = b_s,
    epochs = epochs,
    validation_data = (X_val, Y_val),
    verbose = 2,
    initial_epoch = 0,
    callbacks=[tensorboard_callback]
)


Train on 45000 samples, validate on 5000 samples
Epoch 1/15
198s - loss: 1.4554 - acc: 0.4750 - val_loss: 1.5385 - val_acc: 0.4640
Epoch 2/15
172s - loss: 1.1212 - acc: 0.6025 - val_loss: 1.1597 - val_acc: 0.5830
Epoch 3/15
305s - loss: 0.9660 - acc: 0.6592 - val_loss: 2.0571 - val_acc: 0.4168
Epoch 4/15
167s - loss: 0.8520 - acc: 0.7013 - val_loss: 1.1740 - val_acc: 0.5928
Epoch 5/15
179s - loss: 0.7620 - acc: 0.7328 - val_loss: 1.0888 - val_acc: 0.6164
Epoch 6/15
142s - loss: 0.6799 - acc: 0.7624 - val_loss: 0.9474 - val_acc: 0.6608
Epoch 7/15
185s - loss: 0.6038 - acc: 0.7893 - val_loss: 1.9974 - val_acc: 0.4796
Epoch 8/15
145s - loss: 0.5314 - acc: 0.8148 - val_loss: 9.9184 - val_acc: 0.1422
Epoch 9/15
177s - loss: 0.4644 - acc: 0.8403 - val_loss: 1.1482 - val_acc: 0.6402
Epoch 10/15
143s - loss: 0.3990 - acc: 0.8639 - val_loss: 0.8565 - val_acc: 0.7130
Epoch 11/15
176s - loss: 0.3392 - acc: 0.8848 - val_loss: 1.6802 - val_acc: 0.5710
Epoch 12/15
146s - loss: 0.2832 - acc: 0.9049 -

In [52]:
y_predicted = model.predict_classes(X_test, batch_size=32, verbose=0)
print(y_predicted.shape)
save_predictions('ans1-aas2325', y_predicted)
print(y_predicted[:10])

(10000,)
[3 8 0 7 5 0 3 4 8 1]


In [54]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-aas2325.npy')
print(loaded_y.shape[0])
loaded_y[:10]

10000


array([3, 8, 0, 7, 5, 0, 3, 4, 8, 1])